### Imports 

In [ ]:
%matplotlib inline
from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.applications import imagenet_utils
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import vgg16
from sklearn.metrics import confusion_matrix
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
import pandas as pd
import itertools

### Load & Prep Data

In [ ]:
train_path  = './DATA/SIGN_LANGUAGE/train'
validation_path  = './DATA/SIGN_LANGUAGE/validation'
test_path  = './DATA/SIGN_LANGUAGE/test'

<p>ImageDataGenerator generates batches of tensor image data with real-time data augmentation. 
The data will be looped over (in batches).<br>
    <b>Note:</b> In this example, we won't be doing any image augmentation.</p>

In [ ]:
train_batches = ImageDataGenerator().flow_from_directory(train_path, 
                                                         target_size=(224, 224), 
                                                         batch_size=10)

In [ ]:
validation_batches = ImageDataGenerator().flow_from_directory(validation_path,
                                                         target_size=(224,224), 
                                                         batch_size=30)

In [ ]:
test_batches = ImageDataGenerator().flow_from_directory(test_path, 
                                                        target_size=(224,224), 
                                                        batch_size=50, 
                                                        shuffle=False)

### Load Base Model
#### VGG16 CNN Model pre-trained on ImageNet dataset

In [ ]:
base_model = vgg16.VGG16(weights='imagenet', include_top=False, input_shape = (224,224, 3), pooling='avg')
base_model.summary()

### Freeze the Classification Layers in Base Model

In [ ]:
for layer in base_model.layers[:-5]:
    layer.trainable = False

base_model.summary()

### Define New Model

In [ ]:
# get last layer of the network
last_layer = base_model.get_layer('global_average_pooling2d')

# save the output of the last layer to be the input of the next layer
last_output = last_layer.output

# add our new softmax layer with 3 hidden units
x = Dense(10, activation='softmax', name='softmax')(last_output)

# instantiate a new_model using keras’s Model class
new_model = Model(inputs=base_model.input, outputs=x)

# print the new_model summary
new_model.summary()

### Compile the New Model

In [ ]:
new_model.compile(Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

### Train the New Model

In [ ]:
checkpointer = ModelCheckpoint(filepath='sign-language.model.hdf5', save_best_only=True)

In [ ]:
history = new_model.fit(train_batches, 
                        steps_per_epoch=18, 
                        validation_data=validation_batches, 
                        validation_steps=3, 
                        epochs=20, 
                        verbose=1, 
                        callbacks=[checkpointer])

### Visualize Training

In [ ]:
def plot_training_history(history):          
    hist = pd.DataFrame(history.history)          
    hist['epoch'] = history.epoch          
    plt.figure()          
    plt.xlabel('Epoch')          
    plt.ylabel('Accuracy')          
    plt.plot(hist['epoch'], hist['accuracy'] * 100, label='Train Accuracy')          
    plt.plot(hist['epoch'], hist['val_accuracy'] * 100, label='Validation Accuracy')
    plt.ylim([0, 100])          
    plt.legend()  
    
    plt.figure()          
    plt.xlabel('Epoch')          
    plt.ylabel('Loss')          
    plt.plot(hist['epoch'], hist['loss'], label='Train Loss')          
    plt.plot(hist['epoch'], hist['val_loss'], label='Validation Loss')
    plt.ylim([0,2])
    plt.legend()          
    plt.show()

In [ ]:
plot_training_history(history)

### Evaluate Model